In [1]:
import os
import numpy as np
import nltk
import pandas as pd
import json
from tqdm import tqdm
from matplotlib import pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer
from autocorrect import Speller
from bs4 import BeautifulSoup
from nltk.corpus import stopwords 
from nltk import word_tokenize
import string
import json
import unidecode 
import re 

import warnings
warnings.filterwarnings('ignore')

nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/kian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
dfs = []
for file in os.listdir("./Excels"):
    df = pd.read_excel(os.path.join("./Excels", file))
    dfs.append(df)
    print(file, end=": ")
    print(df.shape)
df = pd.concat(dfs)

science.xlsx: (2080, 19)
JoeBiden.xlsx: (2066, 19)
trump.xlsx: (2080, 19)
nvidia.xlsx: (2010, 19)
Meta.xlsx: (1644, 19)
ISS.xlsx: (2053, 19)
Google.xlsx: (2051, 19)
robots.xlsx: (1982, 19)
Twitter.xlsx: (1616, 19)
abortion.xlsx: (2080, 19)
IranPolitics.xlsx: (2080, 19)
elon.xlsx: (2010, 19)
metaverse.xlsx: (1553, 19)
nasa.xlsx: (2080, 19)
workers.xlsx: (2079, 19)
Microsoft.xlsx: (2080, 19)
democrats.xlsx: (2072, 19)
facebook.xlsx: (2027, 19)
developers.xlsx: (2080, 19)
AbortionBan.xlsx: (2080, 19)
ukraine.xlsx: (2080, 19)
spaceX.xlsx: (2010, 19)
WWDC.xlsx: (2080, 19)
tech.xlsx: (2076, 19)
instagram.xlsx: (1949, 19)
Amazon.xlsx: (2078, 19)
Apple.xlsx: (2031, 19)
war.xlsx: (2079, 19)
AbortionRightsAreHumanRights.xlsx: (2080, 19)
rocket.xlsx: (2056, 19)
Tesla.xlsx: (2004, 19)


In [3]:
df = df[df['Language'] == 'en']
df.reset_index(drop=True, inplace=True)
df.drop_duplicates(subset="Text", keep='first', inplace=True, ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24116 entries, 0 to 24115
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Tweet Id     24116 non-null  int64 
 1   Text         24116 non-null  object
 2   Name         24115 non-null  object
 3   Screen Name  24116 non-null  object
 4   UTC          24116 non-null  object
 5   Created At   24116 non-null  object
 6   Favorites    24116 non-null  int64 
 7   Retweets     24116 non-null  int64 
 8   Language     24116 non-null  object
 9   Client       24116 non-null  object
 10  Tweet Type   24116 non-null  object
 11  URLs         10973 non-null  object
 12  Hashtags     24116 non-null  int64 
 13  Mentions     24116 non-null  int64 
 14  Media Type   8240 non-null   object
 15  Media URLs   8240 non-null   object
 16  Unnamed: 16  913 non-null    object
 17  Unnamed: 17  546 non-null    object
 18  Unnamed: 18  340 non-null    object
dtypes: int64(5), object(14)
m

In [6]:
class Preprocessor:
    def __init__(self):
    
        contraction_map_path = 'HW4/contraction_map.json'
        # contraction_map_path = '/content/gdrive/MyDrive/MIR-HW4/contraction_map.json'
        with open(contraction_map_path) as f:
            self.contraction_map = json.load(f)
        
        nltk.download('stopwords')
        self.stoplist = stopwords.words('english') 
        self.stoplist = set(self.stoplist)
        
        self.w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
        self.lemmatizer = nltk.stem.WordNetLemmatizer()
        
    def remove_newlines_tabs(self, text):
        formatted_text = text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ')
        formatted_text = formatted_text.replace('\\', ' ').replace('. com', '.com')
        return formatted_text

    def remove_mentions_hashtagSigns(self, text):
        tokenized = text.split(" ")
        formatted_text = ""
        for w in tokenized:
            if not w:
                continue
            if w[0] == "@":
                continue
            if w[0] == "#":
                formatted_text += f"{w[1:]} "
            else:
                formatted_text += f"{w} "

        return formatted_text.strip()

    def strip_html_tags(self, text):
        soup = BeautifulSoup(text, "html.parser")
        stripped_text = soup.get_text(separator=" ")
        return stripped_text

    def remove_punctuations(self, text):
        formatted = ""
        for char in text:
            if char in string.punctuation:
                continue
            else:
                formatted += char
        return formatted

    def remove_links(self, text):
        remove_https = re.sub(r'http\S+', '', text)
        remove_com = re.sub(r"\ [A-Za-z]*\.com", " ", remove_https)
        return remove_com

    def remove_whitespace(self, text):
        pattern = re.compile(r'\s+') 
        Without_whitespace = re.sub(pattern, ' ', text)
        text = Without_whitespace.replace('?', ' ? ').replace(')', ') ')
        return text

    def accented_characters_removal(self, text):
        # Remove accented characters from text using unidecode.
        # Unidecode() - It takes unicode data & tries to represent it to ASCII characters. 
        text = unidecode.unidecode(text)
        return text

    def lower_casing_text(self, text):
        return text.lower()

    def reducing_incorrect_character_repeatation(self, text):
        Pattern_alpha = re.compile(r"([A-Za-z])\1{1,}", re.DOTALL)

        # Limiting all the  repeatation to two characters.
        Formatted_text = Pattern_alpha.sub(r"\1\1", text) 

        # Pattern matching for all the punctuations that can occur
        Pattern_Punct = re.compile(r'([.,/#!$%^&*?;:{}=_`~()+-])\1{1,}')

        # Limiting punctuations in previously formatted string to only one.
        Combined_Formatted = Pattern_Punct.sub(r'\1', Formatted_text)

        # The below statement is replacing repeatation of spaces that occur more than two times with that of one occurrence.
        Final_Formatted = re.sub(' {2,}',' ', Combined_Formatted)
        return Final_Formatted


    def expand_contractions(self, text):
        contraction_mapping = self.contraction_map
        list_Of_tokens = text.split(' ')
        for Word in list_Of_tokens: 
             if Word in contraction_mapping: 
                list_Of_tokens = [item.replace(Word, contraction_mapping[Word]) for item in list_Of_tokens]

        String_Of_tokens = ' '.join(str(e) for e in list_Of_tokens) 
        return String_Of_tokens

    def removing_special_characters(self, text):
        formatted_text = re.sub(r"[^a-zA-Z0-9:$-,%.?!]+", ' ', text) 
        return formatted_text

    def remove_numbers(self, text):
        formatted_text = re.sub(r"[^a-zA-Z:$-,%.?!]+", ' ', text)
        return formatted_text

    def removing_stopwords(self, text):
        text = repr(text)
        No_StopWords = [word for word in word_tokenize(text) if word.lower() not in self.stoplist]
        words_string = ' '.join(No_StopWords)    
        return words_string

    def spelling_correction(self, text):
        spell = Speller(lang='en')
        Corrected_text = spell(text)
        return Corrected_text
    
    def cleaning_and_preprocessing(self, tweet,
                              remove_newlines_=True,
                              remove_mentions_hashtagSigns_=True,
                              strip_html_tags_=True,
                              remove_punctuations_=True,
                              remove_links_=True,
                              remove_whitespace_=True,
                              accented_characters_removal_=True,
                              lower_casing_text_=True,
                              reducing_incorrect_character_repeatation_=True,
                              expand_contractions_=True,
                              remove_numbers_=True,
                              removing_stopwords_=True,
                              spelling_correction_=False):
    
        """
        input: a single text (e.g., a tweet or a sentence with type string).
        output: a single text (e.g., a tweet or a sentence with type string) which is clean:)
        """

        ## Cleaning
        text = tweet
        if remove_newlines_:
            text = self.remove_newlines_tabs(text)
        if remove_mentions_hashtagSigns_:
            text = self.remove_mentions_hashtagSigns(text)
        if strip_html_tags_:
            text = self.strip_html_tags(text)
        if remove_punctuations_:
            text = self.remove_punctuations(text)
        if remove_links_:
            text = self.remove_links(text)
        if remove_whitespace_:
            text = self.remove_whitespace(text)
        if  accented_characters_removal_:   
            text = self.accented_characters_removal(text)
        if lower_casing_text_:
            text = self.lower_casing_text(text)
        if reducing_incorrect_character_repeatation_:
            text = self.reducing_incorrect_character_repeatation(text)
        if expand_contractions_:
            text = self.expand_contractions(text)
        if remove_numbers_:
            text = self.remove_numbers(text)

        ## preprocessing
        if removing_stopwords_:
            text = self.removing_stopwords(text)
        if spelling_correction_:
            text = self.spelling_correction(text)
        return text

    def clean_data(self, df):
        """
        gets whole df as input.
        returns df with new column called cleaned tweets.
        """
        cleaned_tweets = []
        for tweet in tqdm(df.Text):
            cleaned_tweet = self.cleaning_and_preprocessing(tweet)
            cleaned_tweets.append(cleaned_tweet)

        df["cleaned_tweets"] = cleaned_tweets
        return df
    
    def lemmatize_data(self, df):
        """
        gets whole df as input.
        returns df with new column called lemmatized tweets.
        """
        lemmatized_tweets = []
        for tweet in tqdm(df.cleaned_tweets):
            lemma = [self.lemmatizer.lemmatize(w,'v') for w in self.w_tokenizer.tokenize(tweet)]
            lemma = ' '.join(lemma)
            lemmatized_tweets.append(lemma)

        df["lemmatized_tweets"] = lemmatized_tweets
        return df
    
    def tokenize_data(self, df):
        pattern = r'''(?x)          # set flag to allow verbose regexps
                (?:[A-Z]\.)+        # abbreviations, e.g. U.S.A.
              | \w+(?:-\w+)*        # words with optional internal hyphens
              | \$?\d+(?:\.\d+)?%?\s?  # currency and percentages, e.g. $12.40, 82%
              | \.\.\.              # ellipsis
              | [][.,;"'?():_`-]    # these are separate tokens; includes ], [
            '''

        pattern = re.compile(pattern)
        nltk_tokenized_tweets = [nltk.regexp_tokenize(tweet, pattern)[1:-1] for tweet in df.lemmatized_tweets]
        df['Text_words'] = nltk_tokenized_tweets
        return df
    
    def perform_clean_lemmatize_tokenize(self, df):
        df = self.clean_data(df)
        df = self.lemmatize_data(df)
        df = self.tokenize_data(df)
        return df
    
    def clean_query(self, text):
        text = self.cleaning_and_preprocessing(text)
        lemma = [self.lemmatizer.lemmatize(w,'v') for w in self.w_tokenizer.tokenize(text)]
        text = ' '.join(lemma)
        pattern = r'''(?x)          # set flag to allow verbose regexps
                (?:[A-Z]\.)+        # abbreviations, e.g. U.S.A.
              | \w+(?:-\w+)*        # words with optional internal hyphens
              | \$?\d+(?:\.\d+)?%?\s?  # currency and percentages, e.g. $12.40, 82%
              | \.\.\.              # ellipsis
              | [][.,;"'?():_`-]    # these are separate tokens; includes ], [
            '''

        pattern = re.compile(pattern)
        return nltk.regexp_tokenize(text, pattern)[1:-1]       

In [7]:
preprocessor = Preprocessor()
df = preprocessor.perform_clean_lemmatize_tokenize(df)
df.head()

[nltk_data] Downloading package stopwords to /Users/kian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24116/24116 [00:02<00:00, 10876.22it/s]


,Tweet Id,Text,Name,Screen Name,UTC,Created At,Favorites,Retweets,Language,Client,...,Hashtags,Mentions,Media Type,Media URLs,Unnamed: 16,Unnamed: 17,Unnamed: 18,cleaned_tweets,lemmatized_tweets,Text_words
0,1541760993121804288,RT @inddais : #inddais is as simple as rocket ...,a happiness seeker.,coucane,2022-06-28T12:30:39.000Z,Tue Jun 28 12:30:39 +0000 2022,0,0,en,"<a href=""http://twitter.com/download/android"" ...",...,2,0,video,https://video.twimg.com/ext_tw_video/153998544...,NaN,NaN,NaN,'rt inddais simple rocket science buy seed har...,'rt inddais simple rocket science buy seed har...,"[rt, inddais, simple, rocket, science, buy, se..."
1,1541760992497000453,@ABC How about not leaving pets and kids in th...,p,therealphattyd,2022-06-28T12:30:39.000Z,Tue Jun 28 12:30:39 +0000 2022,0,0,en,"<a href=""http://twitter.com/download/iphone"" r...",...,0,1,NaN,NaN,NaN,NaN,NaN,'how leaving pets kids car start rocket science ','how leave pet kid car start rocket science ',"[how, leave, pet, kid, car, start, rocket, sci..."
2,1541760990634803202,RT @_demola_ : Science students how far🌚\n\nWh...,onobaby,vibequeen01,2022-06-28T12:30:39.000Z,Tue Jun 28 12:30:39 +0000 2022,0,0,en,"<a href=""http://twitter.com/download/android"" ...",...,0,0,photo,https://pbs.twimg.com/media/FWUj-FnXkAENk1X.jpg,NaN,NaN,NaN,'rt science students far happen litmus paper a...,'rt science students far happen litmus paper a...,"[rt, science, students, far, happen, litmus, p..."
3,1541760984754405376,"RT @DayLabUAB : Wow, congratulations to @DrVic...",Anupam Agarwal,anupamuab,2022-06-28T12:30:37.000Z,Tue Jun 28 12:30:37 +0000 2022,0,0,en,"<a href=""http://twitter.com/download/iphone"" r...",...,0,1,NaN,NaN,NaN,NaN,NaN,'rt wow congratulations new position president...,'rt wow congratulations new position president...,"[rt, wow, congratulations, new, position, pres..."
4,1541760982934237184,RT @mdhn_anbu : Its obvious that studying hist...,Anusha,AnsaMdhn,2022-06-28T12:30:37.000Z,Tue Jun 28 12:30:37 +0000 2022,0,0,en,"<a href=""http://twitter.com/download/android"" ...",...,6,0,NaN,NaN,NaN,NaN,NaN,'rt obvious studying history philosophy much b...,'rt obvious study history philosophy much bett...,"[rt, obvious, study, history, philosophy, much..."


In [ ]:
from sentence_transformers import SentenceTransformer

class TransformerSearch:
    def __init__(df):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.data = df
        
        for tweet in df.Text_words:
            tweet_joined = " ".join(tweet)
            sentence_embeddings = model.encode(sentences)
        


In [12]:
df.to_csv("all.csv")